In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [5]:
results = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc'])

### Data

In [6]:
df = pd.read_excel('./data.xlsx')
df = df.drop(columns=['Company'])
df = df.replace('-', np.nan)
print(df.shape)
df.head()

(899, 827)


,Custom-Return_1390,EPS_MRQ_LC_1390,EPS_TTM_1390,EPS_MRQ_1390,EPS_FY_1390,Rev_TTM_1390,Rev_MRQ_1390,Rev_FY_1390,NI_TTM_1390,NI_MRQ_1390,...,ETR_1401,CFOToRev_1401,CFOToDebt_1401,FCFFToRev_1401,FCFFToNI_1401,FCFEToRev_1401,FCFEToNI_1401,CAPEXToRev_1401,CAPEXToNI_1401,DivYield_1401
0,NaN,11.3750,83.0,11.0,24.0,1.917455e+12,8.399880e+11,1.685499e+12,8.266700e+10,1.137500e+10,...,0.0041,-0.0810,-0.0616,-0.0684,-1.7557,0.0220,0.0689,-0.0123,-0.3158,0.0021
1,0.208274,117.4437,295.0,117.0,295.0,9.046040e+12,2.474342e+12,9.046040e+12,7.951730e+11,3.170980e+11,...,0.1370,0.0267,0.0746,0.0082,0.0930,0.0305,0.3474,0.0187,0.2122,0.0073
2,NaN,88.7262,406.0,89.0,384.0,6.988662e+12,1.702216e+12,6.914857e+12,9.131650e+11,1.996340e+11,...,0.1034,0.0433,0.1469,0.0697,0.5587,0.0757,0.6061,0.0050,0.0402,0.0048
3,-0.481625,-125.1184,-113.0,-125.0,-113.0,2.991100e+10,7.170000e+08,2.991100e+10,-2.570900e+10,-2.852700e+10,...,NaN,-0.5435,-0.0327,-0.6303,NaN,-0.6316,NaN,0.0869,-0.3459,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# convert all columns to float
df = df.astype(float)
df.head()

,Custom-Return_1390,EPS_MRQ_LC_1390,EPS_TTM_1390,EPS_MRQ_1390,EPS_FY_1390,Rev_TTM_1390,Rev_MRQ_1390,Rev_FY_1390,NI_TTM_1390,NI_MRQ_1390,...,ETR_1401,CFOToRev_1401,CFOToDebt_1401,FCFFToRev_1401,FCFFToNI_1401,FCFEToRev_1401,FCFEToNI_1401,CAPEXToRev_1401,CAPEXToNI_1401,DivYield_1401
0,NaN,11.3750,83.0,11.0,24.0,1.917455e+12,8.399880e+11,1.685499e+12,8.266700e+10,1.137500e+10,...,0.0041,-0.0810,-0.0616,-0.0684,-1.7557,0.0220,0.0689,-0.0123,-0.3158,0.0021
1,0.208274,117.4437,295.0,117.0,295.0,9.046040e+12,2.474342e+12,9.046040e+12,7.951730e+11,3.170980e+11,...,0.1370,0.0267,0.0746,0.0082,0.0930,0.0305,0.3474,0.0187,0.2122,0.0073
2,NaN,88.7262,406.0,89.0,384.0,6.988662e+12,1.702216e+12,6.914857e+12,9.131650e+11,1.996340e+11,...,0.1034,0.0433,0.1469,0.0697,0.5587,0.0757,0.6061,0.0050,0.0402,0.0048
3,-0.481625,-125.1184,-113.0,-125.0,-113.0,2.991100e+10,7.170000e+08,2.991100e+10,-2.570900e+10,-2.852700e+10,...,NaN,-0.5435,-0.0327,-0.6303,NaN,-0.6316,NaN,0.0869,-0.3459,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# drop rows which has no value in TSE_FY or RE_FY column
col_tse_fy = []
col_re_fy = []
for col in df.columns:
    if ('TSE_FY' in col):
        col_tse_fy.append(col)
    if ('RE_FY' in col):
        col_re_fy.append(col)
print(len(col_tse_fy), len(col_re_fy))
print(df.shape)
df = df.dropna(subset=col_tse_fy + col_re_fy, how='all')
print(df.shape)

12 12
(899, 827)
(830, 827)


In [9]:
# mean of all col_tse_fy
df['mean_TSE_FY'] = df[col_tse_fy].mean(axis=1)
df['mean_RE_FY'] = df[col_re_fy].mean(axis=1)

# create a y column RE_FY / TSE_FY > .5 = 1 else 0
df['y'] = np.where(df['mean_RE_FY'] / df['mean_TSE_FY'] > .5, 1, 0)
print(df.shape)
print('--------')
print(df[df['y'] >= .5].shape)
print(df[df['y'] < .5].shape)

(830, 830)
--------
(370, 830)
(460, 830)


C:\Users\sydhm\AppData\Local\Temp\ipykernel_6888\628225672.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['mean_TSE_FY'] = df[col_tse_fy].mean(axis=1)
C:\Users\sydhm\AppData\Local\Temp\ipykernel_6888\628225672.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['mean_RE_FY'] = df[col_re_fy].mean(axis=1)
C:\Users\sydhm\AppData\Local\Temp\ipykernel_6888\628225672.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

In [10]:
# print nan proportion
print(df.isna().sum() / df.shape[0])

Custom-Return_1390    0.597590
EPS_MRQ_LC_1390       0.121687
EPS_TTM_1390          0.019277
EPS_MRQ_1390          0.020482
EPS_FY_1390           0.010843
                        ...   
CAPEXToNI_1401        0.081928
DivYield_1401         0.279518
mean_TSE_FY           0.000000
mean_RE_FY            0.006024
y                     0.000000
Length: 830, dtype: float64


In [11]:
# df = df.fillna(df.mean())
# df = df.fillna(df.median())
# df = df.fillna(df.mode().iloc[0])

# fill NaN with 0
df = df.fillna(0)

print(df.isna().sum() / df.shape[0])

Custom-Return_1390    0.0
EPS_MRQ_LC_1390       0.0
EPS_TTM_1390          0.0
EPS_MRQ_1390          0.0
EPS_FY_1390           0.0
                     ... 
CAPEXToNI_1401        0.0
DivYield_1401         0.0
mean_TSE_FY           0.0
mean_RE_FY            0.0
y                     0.0
Length: 830, dtype: float64


In [12]:
# Scaler
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df.head()

,Custom-Return_1390,EPS_MRQ_LC_1390,EPS_TTM_1390,EPS_MRQ_1390,EPS_FY_1390,Rev_TTM_1390,Rev_MRQ_1390,Rev_FY_1390,NI_TTM_1390,NI_MRQ_1390,...,FCFFToRev_1401,FCFFToNI_1401,FCFEToRev_1401,FCFEToNI_1401,CAPEXToRev_1401,CAPEXToNI_1401,DivYield_1401,mean_TSE_FY,mean_RE_FY,y
0,0.042743,0.316850,2.249830e-10,0.999976,0.001849,0.045010,0.055193,0.044945,0.319484,0.495550,...,0.999684,0.893868,0.907533,0.440197,0.483342,0.653700,0.003082,0.360880,0.611768,0.0
1,0.058074,0.320473,2.267615e-10,0.999976,0.001849,0.047014,0.056980,0.047014,0.320198,0.496023,...,0.999685,0.895969,0.907542,0.442539,0.483445,0.657271,0.010713,0.361690,0.612231,0.0
2,0.042743,0.319492,2.276927e-10,0.999976,0.001849,0.046436,0.056136,0.046415,0.320316,0.495841,...,0.999685,0.896498,0.907587,0.444715,0.483399,0.656108,0.007044,0.361677,0.612518,0.0
3,0.007289,0.312188,2.233387e-10,0.999975,0.001849,0.044479,0.054275,0.044479,0.319375,0.495488,...,0.999683,0.895863,0.906876,0.439617,0.483671,0.653497,0.000000,0.360505,0.611575,1.0
4,0.042743,0.330508,2.380534e-10,0.999976,0.001850,0.052995,0.063764,0.052135,0.325979,0.498080,...,0.999685,0.896048,0.907555,0.441255,0.483430,0.656267,0.081450,0.363931,0.615068,1.0


In [13]:
# # duplicate data
# DUPLICATE = 8
# x_train = np.repeat(x_train, DUPLICATE, axis=0)
# y_train = np.repeat(y_train, DUPLICATE, axis=0)
# print(x_train.shape)
# print(y_train.shape)

In [14]:
x_cols = [col for col in df.columns if col != 'y']
x_train, x_test, y_train, y_test = train_test_split(df[x_cols], df['y'], test_size=0.2, random_state=42)

print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

x_train.shape: (664, 829)
x_test.shape: (166, 829)
y_train.shape: (664,)
y_test.shape: (166,)


### Logistic Regression

In [15]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train)

LogisticRegression()

In [16]:
# evaluate the model
y_pred = logistic_model.predict(x_test)

results.loc[len(results)] = [
    'Logistic Regression', 
    round(accuracy_score(y_test, y_pred), 4), 
    round(precision_score(y_test, y_pred), 4), 
    round(recall_score(y_test, y_pred), 4), 
    round(f1_score(y_test, y_pred), 4), 
    round(roc_auc_score(y_test, y_pred), 4)
]
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808


### SVM

In [17]:
svm_model = SVC(kernel='linear', C=1, random_state=42)
svm_model.fit(x_train, y_train)

SVC(C=1, kernel='linear', random_state=42)

In [18]:
y_pred = svm_model.predict(x_test)

results.loc[len(results)] = [
    'SVM',
    round(accuracy_score(y_test, y_pred), 4),
    round(precision_score(y_test, y_pred), 4),
    round(recall_score(y_test, y_pred), 4),
    round(f1_score(y_test, y_pred), 4),
    round(roc_auc_score(y_test, y_pred), 4)
]
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808
1                  SVM    0.8133     0.8393  0.6812  0.7520   0.7942


### Random Forest

In [19]:
rf_model = RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42)
rf_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=4, random_state=42)

In [20]:
y_pred = rf_model.predict(x_test)

results.loc[len(results)] = [
    'Random Forest',
    round(accuracy_score(y_test, y_pred), 4),
    round(precision_score(y_test, y_pred), 4),
    round(recall_score(y_test, y_pred), 4),
    round(f1_score(y_test, y_pred), 4),
    round(roc_auc_score(y_test, y_pred), 4)
]
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808
1                  SVM    0.8133     0.8393  0.6812  0.7520   0.7942
2        Random Forest    0.9940     0.9857  1.0000  0.9928   0.9948


### Sequential Data

In [21]:
# split data
x = df.drop(columns=['y'])
y = df['y']
x = np.array(x)
y = np.array(y)
# [samples, timesteps, features]
x = x.reshape(x.shape[0], 1, x.shape[1])

x_train = x[:int(x.shape[0] * .7)]
y_train = y[:int(y.shape[0] * .7)]
x_valid = x[int(x.shape[0] * .7):int(x.shape[0] * .85)]
y_valid = y[int(y.shape[0] * .7):int(y.shape[0] * .85)]
x_test = x[int(x.shape[0] * .85):]
y_test = y[int(y.shape[0] * .85):]

print(x_train.shape)
print(y_train.shape)
print("------")
print(x_valid.shape)
print(y_valid.shape)
print("------")
print(x_test.shape)
print(y_test.shape)

(581, 1, 829)
(581,)
------
(124, 1, 829)
(124,)
------
(125, 1, 829)
(125,)


### LSTM

In [22]:
# LSTM
lstm = keras.Sequential()
lstm.add(keras.layers.LSTM(256, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
lstm.add(keras.layers.Dropout(0.1))
lstm.add(keras.layers.LSTM(256, return_sequences=False))
lstm.add(keras.layers.Dropout(0.1))
lstm.add(keras.layers.Dense(1, activation='sigmoid'))
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 256)            1112064   
                                                                 
 dropout (Dropout)           (None, 1, 256)            0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 1637633 (6.25 MB)
Trainable params: 1637633 (6.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# train
lstm_history = lstm.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_valid, y_valid), shuffle=False)

Epoch 1/100
10/10 [==============================] - 13s 205ms/step - loss: 0.6987 - accuracy: 0.5456 - val_loss: 0.6968 - val_accuracy: 0.4516
Epoch 2/100
10/10 [==============================] - 0s 38ms/step - loss: 0.6991 - accuracy: 0.4630 - val_loss: 0.6885 - val_accuracy: 0.5484
Epoch 3/100
10/10 [==============================] - 0s 39ms/step - loss: 0.6937 - accuracy: 0.5628 - val_loss: 0.6932 - val_accuracy: 0.5484
Epoch 4/100
10/10 [==============================] - 0s 37ms/step - loss: 0.6893 - accuracy: 0.5628 - val_loss: 0.6908 - val_accuracy: 0.5484
Epoch 5/100
10/10 [==============================] - 0s 41ms/step - loss: 0.6861 - accuracy: 0.5628 - val_loss: 0.6887 - val_accuracy: 0.5484
Epoch 6/100
10/10 [==============================] - 0s 38ms/step - loss: 0.6860 - accuracy: 0.5628 - val_loss: 0.6875 - val_accuracy: 0.5484
Epoch 7/100
10/10 [==============================] - 0s 37ms/step - loss: 0.6857 - accuracy: 0.5628 - val_loss: 0.6859 - val_accuracy: 0.5484
Epoc

In [24]:
# evaluate the model
y_pred = lstm.predict(x_test)

4/4 [==============================] - 1s 9ms/step


In [25]:
# calc metrics and add a row to results
results.loc[len(results)] = [
    'LSTM',
    round(accuracy_score(y_test, y_pred.round()), 4),
    round(precision_score(y_test, y_pred.round()), 4),
    round(recall_score(y_test, y_pred.round()), 4),
    round(f1_score(y_test, y_pred.round()), 4),
    round(roc_auc_score(y_test, y_pred.round()), 4),
]
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808
1                  SVM    0.8133     0.8393  0.6812  0.7520   0.7942
2        Random Forest    0.9940     0.9857  1.0000  0.9928   0.9948
3                 LSTM    0.7120     0.6500  0.8667  0.7429   0.7179


In [26]:
# plot loss
fig = go.Figure()
fig.add_trace(go.Scatter(y=lstm_history.history['loss'], mode='lines', name='loss'))
fig.add_trace(go.Scatter(y=lstm_history.history['val_loss'], mode='lines', name='val_loss'))
fig.update_layout(title='Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [27]:
# plot accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(y=lstm_history.history['accuracy'], mode='lines', name='accuracy'))
fig.add_trace(go.Scatter(y=lstm_history.history['val_accuracy'], mode='lines', name='val_accuracy'))
fig.update_layout(title='Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

### RNN

In [28]:
rnn_model = keras.Sequential()
rnn_model.add(keras.layers.SimpleRNN(256, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
rnn_model.add(keras.layers.Dropout(0.1))
rnn_model.add(keras.layers.SimpleRNN(256, return_sequences=False))
rnn_model.add(keras.layers.Dropout(0.1))
rnn_model.add(keras.layers.Dense(1, activation='sigmoid'))
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 1, 256)            278016    
                                                                 
 dropout_2 (Dropout)         (None, 1, 256)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 409601 (1.56 MB)
Trainable params: 409601 (1.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
# train
rnn_history = rnn_model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_valid, y_valid), shuffle=False)

Epoch 1/100
10/10 [==============================] - 3s 59ms/step - loss: 0.8221 - accuracy: 0.5353 - val_loss: 0.7732 - val_accuracy: 0.4516
Epoch 2/100
10/10 [==============================] - 0s 15ms/step - loss: 0.8266 - accuracy: 0.4905 - val_loss: 0.7858 - val_accuracy: 0.4516
Epoch 3/100
10/10 [==============================] - 0s 13ms/step - loss: 0.8045 - accuracy: 0.4664 - val_loss: 0.6895 - val_accuracy: 0.4839
Epoch 4/100
10/10 [==============================] - 0s 13ms/step - loss: 0.7257 - accuracy: 0.5043 - val_loss: 0.6805 - val_accuracy: 0.5484
Epoch 5/100
10/10 [==============================] - 0s 14ms/step - loss: 0.7330 - accuracy: 0.5198 - val_loss: 0.6896 - val_accuracy: 0.4516
Epoch 6/100
10/10 [==============================] - 0s 14ms/step - loss: 0.7343 - accuracy: 0.5112 - val_loss: 0.6758 - val_accuracy: 0.6613
Epoch 7/100
10/10 [==============================] - 0s 14ms/step - loss: 0.7124 - accuracy: 0.5422 - val_loss: 0.6808 - val_accuracy: 0.5403
Epoch 

In [30]:
# evaluate the model
y_pred = rnn_model.predict(x_test)

4/4 [==============================] - 0s 3ms/step


In [31]:
# calc metrics and add a row to results
results.loc[len(results)] = [
    'RNN',
    round(accuracy_score(y_test, y_pred.round()), 4),
    round(precision_score(y_test, y_pred.round()), 4),
    round(recall_score(y_test, y_pred.round()), 4),
    round(f1_score(y_test, y_pred.round()), 4),
    round(roc_auc_score(y_test, y_pred.round()), 4),
]
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808
1                  SVM    0.8133     0.8393  0.6812  0.7520   0.7942
2        Random Forest    0.9940     0.9857  1.0000  0.9928   0.9948
3                 LSTM    0.7120     0.6500  0.8667  0.7429   0.7179
4                  RNN    0.6880     0.6207  0.9000  0.7347   0.6962


In [32]:
# plot loss
fig = go.Figure()
fig.add_trace(go.Scatter(y=lstm_history.history['loss'], mode='lines', name='loss'))
fig.add_trace(go.Scatter(y=lstm_history.history['val_loss'], mode='lines', name='val_loss'))
fig.update_layout(title='Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

In [33]:
# plot accuracy
fig = go.Figure()
fig.add_trace(go.Scatter(y=lstm_history.history['accuracy'], mode='lines', name='accuracy'))
fig.add_trace(go.Scatter(y=lstm_history.history['val_accuracy'], mode='lines', name='val_accuracy'))
fig.update_layout(title='Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

### Results

In [34]:
print(results)

                 model  accuracy  precision  recall      f1  roc_auc
0  Logistic Regression    0.7952     0.7869  0.6957  0.7385   0.7808
1                  SVM    0.8133     0.8393  0.6812  0.7520   0.7942
2        Random Forest    0.9940     0.9857  1.0000  0.9928   0.9948
3                 LSTM    0.7120     0.6500  0.8667  0.7429   0.7179
4                  RNN    0.6880     0.6207  0.9000  0.7347   0.6962


In [35]:
# plot comparative metrics for different models



In [36]:
# plot roc_auc by models
# x, y from 0 to 1

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1],
                    mode='lines',
                    name='baseline'))
fig.add_trace(go.Scatter(x=[0, 1], y=[0.5, 0.5],
                    mode='lines',
                    name='baseline'))
fig.add_trace(go.Scatter(x=[0, 1], y=[1, 1],
                    mode='lines',
                    name='baseline'))

# add roc_auc from results df
for model in results['model'].unique():
    df = results[results['model'] == model]
    fig.add_trace(go.Scatter(x=results['roc_auc'], y=results['roc_auc'],
                    mode='lines',
                    name=model))

fig.update_layout(title='ROC-AUC',
                     xaxis_title='False Positive Rate',
                     yaxis_title='True Positive Rate')
fig.show()


In [26]:
df1 = pd.read_excel('./data.xlsx')
df1 = df1.drop(columns=['Company'])
df1 = df1.replace('-', np.nan)
df1 = df1.astype(float)

Columns = [column for column in df1.columns if "_1390" in column]
for i in range(len(Columns)):
    Columns[i] = Columns[i].replace("_1390", "")

mean_columns = {}
for i in Columns:
    my_list = []
    for j in df1.columns:
        if i in j :
            my_list.append(j)
    mean = df1[my_list].mean(axis=1, skipna=True).tolist()
    mean_columns[f'mean_{i}'] = mean
    df1.drop(columns=my_list, inplace=True)
df_means = pd.DataFrame(mean_columns)
df1 = pd.concat([df1, df_means], axis=1)
print(df1.describe())


       mean_Custom-Return  mean_EPS_MRQ_LC  mean_EPS_TTM  mean_EPS_MRQ  \
count          717.000000       732.000000  8.160000e+02  8.150000e+02   
mean           285.134415       565.309521  1.460786e+11 -1.001601e+06   
std           6228.415337      1516.729680  4.172839e+12  2.860917e+07   
min             -0.863708     -9265.575400 -2.708508e+04 -8.167400e+08   
25%              0.567673        42.648400  1.956250e+02  3.800000e+01   
50%              0.792328       205.712250  6.523333e+02  1.960000e+02   
75%              1.093658       598.376100  1.682250e+03  5.665417e+02   
max         163300.044086     20013.094500  1.192001e+14  2.001300e+04   

        mean_EPS_FY  mean_Rev_TTM  mean_Rev_MRQ   mean_Rev_FY   mean_NI_TTM  \
count  8.240000e+02  7.710000e+02  7.790000e+02  7.900000e+02  8.060000e+02   
mean   2.798720e+06  5.390644e+13  1.505712e+13  5.149155e+13  1.330690e+13   
std    8.044181e+07  2.259925e+14  6.040528e+13  2.253554e+14  5.686283e+13   
min   -4.277600e+